## Set Up



In this notebook, I will be exploring the three approaches of presenting SQL in a Jupyter Notebook demonstrated in [lecture 20 from DATA 100](https://ds100.org/fa24/resources/assets/lectures/lec20/lec20.html). The three approaches are SQL Magic, `pandas`, and DuckDB.

In [1]:
#pip install jupysql --upgrade
#%pip install duckdb-engine --quiet

In [2]:
import sqlite3
import duckdb
import pandas as pd
%load_ext sql

In [3]:
# Create a DuckDB database file (example.duckdb)
conn = duckdb.connect('example_duck.db')

# Optional: Create a table
conn.execute('''
    CREATE TABLE IF NOT EXISTS Dragon (
        name TEXT PRIMARY KEY,
        year INTEGER,
        cute INTEGER
    )
''')

# Optional: Insert data
conn.execute('''
    INSERT INTO Dragon (name, year, cute) 
    VALUES('hiccup', 2010, 10),
    ('drogon', 2011, -100),
    ('dragon 2', 2019, 0),
    ('puff', 2010, 100),
    ('smaug', 2011, NULL)
''')

# Save changes and close the connection
conn.close()


Need to connect to duckdb.

In [4]:
%sql duckdb:///example_duck.db

Connecting to 'duckdb:///example_duck.db'

In [5]:
%%sql 
SELECT * 
FROM Dragon;

Running query in 'duckdb:///example_duck.db'

name,year,cute
hiccup,2010,10
drogon,2011,-100
dragon 2,2019,0
puff,2010,100
smaug,2011,None


Using `pandas` to read SQL queries.

In [6]:
import sqlalchemy 
import pandas as pd

engine = sqlalchemy.create_engine("duckdb:///example_duck.db")

In [7]:
query = """
SELECT * 
FROM Dragon;
"""

df = pd.read_sql(query, engine)
df

,name,year,cute
0,hiccup,2010,10.0
1,drogon,2011,-100.0
2,dragon 2,2019,0.0
3,puff,2010,100.0
4,smaug,2011,NaN


## Practicing SQL
### Financial Data
I am interested in fiance, and to get started, I will be using this [financial data](https://datahub.io/core/s-and-p-500-companies-financials). There are two datasets in this source. The first one is called `constituents` and the other one is called `constituents-financial`.

According to [S&P 500](https://www.spglobal.com/spdji/en/indices/equity/sp-500/#overview), they are used to gauge the large-cap U.S. equities. They document their methodology and have their index include 500 leading companies and covers approximately 80% of available market capitalization. The dataset is currently not published on their website, so I will be using datahub.io.


In [8]:
conn = duckdb.connect('finance_database.db')

# Optional: Create a table
conn.execute('''
    CREATE TABLE IF NOT EXISTS constituents AS SELECT * FROM read_csv_auto('constituents.csv')
''')

conn.execute('''
    CREATE TABLE IF NOT EXISTS constituents_financial AS SELECT * FROM read_csv_auto('constituents-financial.csv')
''')

conn.close()

In [9]:
%sql duckdb:///finance_database.db

Connecting and switching to connection 'duckdb:///finance_database.db'

In [10]:
%%sql 
SELECT * 
FROM constituents;

Running query in 'duckdb:///finance_database.db'

Symbol,Name,Sector
A,Agilent Technologies,Life Sciences Tools & Services
AAPL,Apple Inc.,"Technology Hardware, Storage & Peripherals"
ABBV,AbbVie,Biotechnology
ABNB,Airbnb,"Hotels, Resorts & Cruise Lines"
ABT,Abbott Laboratories,Health Care Equipment
ACGL,Arch Capital Group,Property & Casualty Insurance
ACN,Accenture,IT Consulting & Other Services
ADBE,Adobe Inc.,Application Software
ADI,Analog Devices,Semiconductors
ADM,Archer Daniels Midland,Agricultural Products & Services


In [11]:
%%sql 
SELECT * 
FROM constituents_financial;

Running query in 'duckdb:///finance_database.db'

Symbol,Name,Sector,Price,Price/Earnings,Dividend Yield,Earnings/Share,52 Week Low,52 Week High,Market Cap,EBITDA,Price/Sales,Price/Book,SEC Filings
A,Agilent Technologies,Life Sciences Tools & Services,151.52,34.126125,null,null,124.16,155.35,43273355264,null,null,null,http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=A
AAPL,Apple Inc.,"Technology Hardware, Storage & Peripherals",236,37.76,0.0042,6.25,164.08,260.1,3572851277824,134660997120,9.1369095,62.649323,http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=AAPL
ABBV,AbbVie,Biotechnology,183.9,64.3007,0.0373,2.86,153.58,207.32,324977033216,25629999104,5.8519626,53.88221,http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=ABBV
ABNB,Airbnb,"Hotels, Resorts & Cruise Lines",131.17,45.387543,null,2.89,110.38,170.1,81951334400,1672999936,7.560086,9.859441,http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=ABNB
ABT,Abbott Laboratories,Health Care Equipment,127.93,16.744764,0.0202,7.64,99.71,129.85,221889478656,10824999936,5.3834457,5.5757494,http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=ABT
ACGL,Arch Capital Group,Property & Casualty Insurance,93.07,6.3485675,null,14.66,80.97,116.47,35584942080,5171999744,2.1097374,1.6261313,http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=ACGL
ACN,Accenture,IT Consulting & Other Services,384.95,32.294464,0.017,11.92,278.69,387.51,240778526720,11065912320,3.7101948,8.508687,http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=ACN
ADBE,Adobe Inc.,Application Software,437.45,35.42105,null,12.35,403.75,638.25,190421991424,8134000128,9.090657,13.383815,http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=ADBE
ADI,Analog Devices,Semiconductors,211.89,64.798164,0.0165,3.27,182.57,244.14,105128370176,4361349120,10.837489,2.9895453,http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=ADI
ADM,Archer Daniels Midland,Agricultural Products & Services,51.23,14.39045,0.0379,3.56,48.4,66.08,24515297280,3988000000,0.27617267,1.0890499,http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=ADM
